In [1]:
# 12/11(목) 11:08
# 강사님 거 보고 오류난 거 해결하기.

# Transformers AutoClass 

### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading
    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델 자동으로 로드.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델 로드.
    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드.
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델 생성할 수있다.
       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`
- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 별 estimator layer를 추가한 모델 생성해 제공.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델 자동으로 로드.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`
        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델 자동으로 로드.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델 자동으로 로드.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

In [2]:
from transformers import AutoModel, AutoConfig, AutoTokenizer

model_id = "bert-base-uncased"
# model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer))

model = AutoModel.from_pretrained(model_id)
print(type(model))

config = AutoConfig.from_pretrained(model_id)
print(type(config))

<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.bert.configuration_bert.BertConfig'>


In [3]:
# config.json - 모델 아키텍쳐 관련 메타정보파일
# config
model2 = AutoModel.from_config(config) # 학습이 안 된 모델
type(model2)

transformers.models.bert.modeling_bert.BertModel

In [4]:
from transformers import BertModel
b_model = BertModel.from_pretrained(model_id)
b_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [5]:
#####################
# raw-text -> (Tokenizer) ==> token ids ==> (model) ==> 결과 

In [6]:
# Tokenizer
raw_text = "I am a boy."

# 토큰화
token = tokenizer(
    raw_text,
    return_tensors='pt'
    )

token

{'input_ids': tensor([[ 101, 1045, 2572, 1037, 2879, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [7]:
# token (결과)의 key들 조회
dict(token).keys()
# 결과: input_ids': 입력 토큰 id값. 
# 'token_type_ids': 입력이 문장 쌍일 때 문장을 구분하는 id. 0: 첫 번째 문장, 1: 두 번째 문장. 
#                   각 토큰이 어느 문장에 속했는지 토큰 별로 지정됨. 
#                   문장 쌍: QA(질문-지문)
# 'attention_mask': 실제 문장이 토큰과 Padding을 구분한다. 토큰 위치 별로 지정. 0: padding, 1: 

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [8]:
token['input_ids']

tensor([[ 101, 1045, 2572, 1037, 2879, 1012,  102]])

In [9]:
type(token['input_ids']), type(token['token_type_ids']), type(token['attention_mask'])

(torch.Tensor, torch.Tensor, torch.Tensor)

In [10]:
###################### 
# token 결과 -> model

# keyword 가변 인자에 맞춰서 name = value 형태로 입력. input_ids=[.....],
# model에 맞춤 타입으로 입력. (pytorch 모델: torch.tensor)
context_vector = model(**token) # BertModel -> feature extract

In [11]:
context_vector.last_hidden_state.shape
# 모든 입력 토큰들의 hidden state들
# [batch:1, seq_len: 7, hidden_size(embedding vector): 768]

torch.Size([1, 7, 768])

In [12]:
# 입력 문장 (문서)에 대한 context vector - 문장(문서)의 특성값. 
## Bert는 last_hidden_state에서 첫 번째 토큰 값을 가공(특성 추출)해서 context vector로 사용.
token['input_ids'].shape

torch.Size([1, 7])

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저, 모델 로드

In [13]:
from transformers import AutoModel, AutoTokenizer

model_id = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

In [14]:
type(tokenizer), type(model)

(transformers.models.bert.tokenization_bert_fast.BertTokenizerFast,
 transformers.models.bert.modeling_bert.BertModel)

In [15]:
# 단일 문서: padding - pad 추가, truncation - 잘라내기 (토큰 수 맞추는 작업) 신경 쓸 필요 없다.
tokens = tokenizer(
    "나는 어제 친구와 밥을 먹었다.",
    return_tensors="pt"
)
# tokenizer(토큰화 할 문장): 모델에 입력할 수 있는 형태로 토큰 결과를 반환.
tokens

{'input_ids': tensor([[    2,  8616,  9909,  9025,  4196, 23905, 21452,  4020,    17,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [16]:
tokenizer.encode("나는 어제 친구와 밥을 먹었다.")  # input_id만 출력.
tokenizer.decode([2, 8616, 9909, 9025, 4196, 23905, 21452, 4020, 17, 3]) # token들 -> 문장 
tokenizer.decode(
    [2, 8616, 9909, 9025, 4196, 23905, 21452, 4020, 17, 3],
    skip_special_tokens=True    # 스페셜 토큰은 제거.
)

'나는 어제 친구와 밥을 먹었다.'

In [49]:
# 개별 토큰 관련 조회
tokenizer.convert_ids_to_tokens(8616)  # 토큰 ID -> 토큰 문자열
tokenizer.convert_ids_to_tokens([2, 8616, 9909]) 

tokenizer.convert_tokens_to_ids("어제") 
tokenizer.convert_tokens_to_ids(["어제", "너는", "나를"]) 


[9909, 9039, 15919]

In [50]:
# 토크나이저 전체 어휘 수
print(tokenizer.vocab_size)
tokenizer.get_vocab()   # 어휘사전 조회

30000


{'##자루': 24907,
 '##자빠졌네': 22765,
 '위안부': 9319,
 '##🦆': 7373,
 '끝났': 12633,
 '인성이': 14756,
 '##땀': 5714,
 '하다하다': 14335,
 '##내': 4292,
 '닻': 804,
 '3': 22,
 '##등을': 17750,
 '어처구니': 13401,
 '##냨': 4662,
 '⚠': 133,
 '진정한': 9407,
 '감염이': 27547,
 '멓': 1354,
 '대치': 28094,
 '판단은': 26386,
 '##것들도': 25205,
 '주무': 22970,
 '##먺': 7049,
 '##흠': 4935,
 '불과': 11674,
 '##옹호': 21823,
 '##할지': 12094,
 '안좋은': 15827,
 '마찬가지다': 21063,
 '##줍': 4661,
 '##보다가': 23517,
 '의뢰': 25433,
 '드러운': 16019,
 '넘길': 26769,
 '##머니': 8489,
 '##하구': 12526,
 '나옴': 15599,
 '애들': 8650,
 '알았는데': 13193,
 '못': 1391,
 '없는': 8106,
 '틀림': 16441,
 '올라가는': 26922,
 '반성도': 24600,
 '🐂': 3668,
 '##뺏': 4823,
 '굷': 356,
 '세상이': 9863,
 '실망이다': 24330,
 '워': 2376,
 '##재팬': 22694,
 '##하지않': 9881,
 '무혐의': 21349,
 '##덩어': 25778,
 '나가리': 25872,
 '주문': 15681,
 '##외교': 12513,
 '사실에': 25913,
 '밢': 1490,
 '미용': 27846,
 '노란': 14872,
 '외치던': 19801,
 '##은혜': 19485,
 '체육': 19722,
 '일으': 10066,
 '불태': 23134,
 '쓩': 2138,
 '강대': 26877,
 '##미디': 12312,
 '##라

### 입력값 토큰화

In [19]:
sentences = [
    "안녕",
    "Hugging Face는 인공지능(AI)과 자연어 처리(NLP) 분야에서 혁신적인 도구와 모델을 제공하는 AI 스타트업이다.",
    "2016년에 설립된 이 회사는 주로 오픈소스 라이브러리와 사전 학습된 NLP 모델을 제공을 제공한다."
]

In [20]:
sent_tokens = tokenizer(
    sentences
    # return_tensors="pt"
)

In [21]:
print(len(sent_tokens['input_ids']), sent_tokens['input_ids'])
print(len(sent_tokens['attention_mask']))
print(len(sent_tokens['token_type_ids']))

3 [[2, 19017, 3], [2, 41, 4223, 4403, 4403, 18940, 39, 4243, 4773, 4226, 4008, 14583, 25061, 11, 22502, 12, 321, 10459, 4071, 9810, 11, 47, 4450, 4579, 12, 16029, 7971, 13064, 8097, 867, 4228, 4196, 16505, 4027, 13248, 7966, 22502, 12296, 4104, 22192, 4020, 17, 3], [2, 26182, 4113, 20684, 4130, 2451, 22088, 20002, 15999, 4266, 4103, 17564, 4408, 4053, 4038, 4196, 11202, 20323, 4130, 47, 4450, 4579, 16505, 4027, 13248, 4027, 13248, 8008, 17, 3]]
3
3


In [ ]:
# max_length = 최대 길이 (토큰 수)를 지정. 
# truncation - 길이를 맞추기 위해서 토큰의 일부를 잘라내는 것.
##           - True("longest"): 배치 중에서 가장 긴 문서에 맞춘다. max_length 설정이 된 경우에는 max_length에 맞춘다. 
##           - False("do_not_truncate"): 기본값. truncation을 안 함.

# padding - 길이를 맞추기 위해서 [PAD] 토큰으로 채우는 것.
##        - True("longest"): 배치 중에 가장 긴 문서에 맞춘다. 
##        - "max_length": max_length 설정에 맞춘다. (max_length가 없으면 model의 default max_length에 맞춘다.)
##        - False("do_not_pad"): 기본값. Padding 처리하지 않는다. 

# 순서: truncation -> padding


In [23]:
# 토큰 수
for ids in sent_tokens['input_ids']:
    print(len(ids))

3
43
30


In [56]:
sent_tokens2 = tokenizer(
    sentences, 
    max_length=20,
    padding=True,    # 붙이는 것
    truncation=True  # 잘라내는 것
)

In [57]:
for sent in sent_tokens2['input_ids']:
    print(len(sent))  #, sent)

20
20
20


In [58]:
sent_tokens2['attention_mask'][0]

[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [59]:
context_vector = tokenizer(
    sentences, 
    max_length=20,
    padding=True,    # 붙이는 것
    truncation=True,  # 잘라내는 것
    return_tensors="pt"
)

In [60]:
result = model(**tokens)

In [ ]:
result.last_hidden_state.shape  # torch.Size([1: 문서 개수, 10: seq_len(토큰 수), 768])


torch.Size([1, 10, 768])

In [62]:
result.pooler_output.shape

torch.Size([1, 768])